Notebook to do a direct comparison between the PSF second moments published in the ConsDB vs the DRP to make sure they are not very divergent

In [ ]:
from lsst.summit.utils import ConsDbClient

In [ ]:
import numpy as np
from astropy.table import Table, join
from astropy.time import Time

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
%matplotlib widget

from lsst.meas.algorithms.installGaussianPsf import FwhmPerSigma

from tqdm.notebook import tqdm

In [ ]:
def getAirmassSeeingCorrection(airmass: float) -> float:
    """Get the correction factor for seeing due to airmass.

    Parameters
    ----------
    airmass : `float`
        The airmass, greater than or equal to 1.

    Returns
    -------
    correctionFactor : `float`
        The correction factor to apply to the seeing.

    Raises
    ------
        ValueError raised for unphysical airmasses.
    """
    if airmass < 1:
        raise ValueError(f"Invalid airmass: {airmass}")
    return airmass ** (-0.6)

def getBandpassSeeingCorrection(filterName: str) -> float:
    """Get the correction factor for seeing due to a filter.

    Parameters
    ----------
    filterName : `str`
        The name of the filter, e.g. 'SDSSg_65mm'.

    Returns
    -------
    correctionFactor : `float`
        The correction factor to apply to the seeing.

    Raises
    ------
        ValueError raised for unknown filters.
    """
    match filterName:
        case "SDSSg_65mm":  # LATISS
            return (474.41 / 500.0) ** 0.2
        case "SDSSr_65mm":  # LATISS
            return (628.47 / 500.0) ** 0.2
        case "SDSSi_65mm":  # LATISS
            return (769.51 / 500.0) ** 0.2
        case "SDSSz_65mm":  # LATISS
            return (871.45 / 500.0) ** 0.2
        case "SDSSy_65mm":  # LATISS
            return (986.8 / 500.0) ** 0.2
        case "u_02":  # ComCam
            return (370.697 / 500.0) ** 0.2
        case "g_01":  # ComCam
            return (476.359 / 500.0) ** 0.2
        case "r_03":  # ComCam
            return (619.383 / 500.0) ** 0.2
        case "i_06":  # ComCam
            return (754.502 / 500.0) ** 0.2
        case "z_03":  # ComCam
            return (866.976 / 500.0) ** 0.2
        case "y_04":  # ComCam
            return (972.713 / 500.0) ** 0.2
        case _:
            raise ValueError(f"Unknown filter name: {filterName}")

In [ ]:
os.environ["no_proxy"] += ",.consdb"

In [ ]:
url="http://consdb-pq.consdb:8080/consdb"

In [ ]:
consdb=ConsDbClient(url)

In [ ]:
# Query both consDB tables
exposure = consdb.query("SELECT * FROM cdb_lsstcomcam.exposure WHERE science_program = 'BLOCK-320'")
visits = consdb.query("SELECT * FROM cdb_lsstcomcam.visit1 WHERE science_program = 'BLOCK-320'")
visits_ql = consdb.query("SELECT * FROM cdb_lsstcomcam.visit1_quicklook")

# Join using astropy's join function on 'visit_id'
exposure_join = exposure.rename_column("exposure_id", "visit_id")
merged_exposure = join(exposure, visits, keys="visit_id", join_type="inner")  
merged_visits = join(visits, visits_ql, keys="visit_id", join_type="inner")  

# Display or use the merged table
print(merged_visits)

In [ ]:
merged_visits.colnames

In [ ]:
# Convert PSF sigma to FWHM
sig2fwhm = 2 * np.sqrt(2 * np.log(2))
pixel_scale = 0.2  # arcsec / pixel
merged_visits["psf_fwhm"] = merged_visits["psf_sigma_median"] * sig2fwhm * pixel_scale

# Add the FWHM at zenith at 500nm
merged_visits["fwhm_zenith_500nm"] = [
    fwhm * getBandpassSeeingCorrection(filt) * getAirmassSeeingCorrection(airmass)
    for fwhm, filt, airmass in zip(merged_visits["psf_fwhm"], merged_visits["physical_filter"], merged_visits["airmass"])
]

In [ ]:
time = Time(merged_visits['exp_midpt'])

# Plot the time vs the specified column
plt.figure(figsize=(10, 6))
plt.plot(time.plot_date, merged_visits['psf_fwhm'])

# Set x-axis to show dates
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())

# Rotate and format x-axis labels for readability
plt.xticks(rotation=45, ha='right')

plt.xlabel('Time')
plt.ylabel('FWHM (arcsec)')
plt.title(f'Time vs PSF FWHM zenith 500nm median')
plt.legend()
plt.grid(True)
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()

In [ ]:
# Load the CSV file into an Astropy Table
file_path = "ringss4rubin.csv"
ringss_data = Table.read(file_path, format="csv")

# Display table info to check structure
ringss_data.info()

In [ ]:
column_name='see'

# Convert time column to Astropy Time object
time_column = ringss_data.columns[0]
time = Time(time_column, format='iso')

# Get the data from the specified column
data = ringss_data[column_name]

# Plot the time vs the specified column
plt.figure(figsize=(10, 6))
plt.plot(time.plot_date, data, label=column_name)

# Set x-axis to show dates
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())

# Rotate and format x-axis labels for readability
plt.xticks(rotation=45, ha='right')

plt.xlabel('Time')
plt.ylabel(column_name)
plt.title(f'Time vs {column_name}')
plt.legend()
plt.grid(True)
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()

In [ ]:
# Ensure time columns are Astropy Time objects
ringss_data["time"] = Time(ringss_data["time"])
merged_visits["exp_midpt"] = Time(merged_visits["exp_midpt"])

# Find the nearest neighbor index in 'table' for each entry in 'ringss_data'
idx = np.searchsorted(merged_visits["exp_midpt"].jd, ringss_data["time"].jd)

# Prevent out-of-bounds indices
idx = np.clip(idx, 0, len(merged_visits) - 1)

# Compute time differences (keeping TimeDelta)
time_diffs = merged_visits["exp_midpt"][idx] - ringss_data["time"]

# Extract seconds properly **before using NumPy operations**
time_diffs_sec = np.abs(time_diffs.sec)  # Now it's a NumPy array of seconds

# Define the max time difference threshold
max_diff = 30  # in seconds
mask = time_diffs_sec < max_diff  # Now this will work correctly

# Merge tables using the matched indices
merged_table = ringss_data[mask].copy()  # Copy only matched rows
for col in merged_visits.colnames:
    merged_table[col] = merged_visits[col][idx][mask]  # Copy matched columns

# Print merged table
print(merged_table)

In [ ]:
sqr_diff = merged_table["fwhm_zenith_500nm"]**2.0 - merged_table["see"]**2.0

In [ ]:
# Plot histogram using Freedman-Diaconis rule
plt.figure()
plt.xlabel('Seeing')
plt.ylabel("Frequency")
plt.title(f"ComCam Delivered IQ - RINGSS Seeing during ComCam On-Sky")
plt.grid(True, linestyle="--", alpha=0.6, zorder=0)
plt.hist(merged_table["fwhm_zenith_500nm"], bins='fd', edgecolor='0.1', zorder=2,alpha=0.5, label='ComCam FWHM')
plt.hist(merged_table["see"], bins="fd", edgecolor='0.1', zorder=3,alpha=0.5, label='RINGSS')
plt.legend()
plt.show()

In [ ]:
# Plot histogram using Freedman-Diaconis rule
plt.figure()
plt.xlabel('Seeing')
plt.ylabel("Frequency")
plt.title(f"ComCam Delivered IQ - RINGSS Seeing during ComCam On-Sky")
plt.grid(True, linestyle="--", alpha=0.6, zorder=0)
plt.hist(merged_table["see2"], bins='fd', edgecolor='0.1', zorder=2,alpha=0.5, label='RINGSS Profile-Weighted')
plt.hist(merged_table["see"], bins="fd", edgecolor='0.1', zorder=3,alpha=0.5, label='RINGSS')
plt.legend()
plt.show()

In [ ]:
# Plot the time vs the specified column
plt.figure(figsize=(10, 6))
plt.plot(merged_table["see"], merged_table["fwhm_zenith_500nm"],'.', color='0.5',zorder=0, alpha=0.5)
sns.kdeplot(
    x=merged_table["see"], 
    y=merged_table["fwhm_zenith_500nm"], 
    levels=10,  # Number of contour levels
    cmap="Reds",  # Color map for contours
    alpha=0.6,
    zorder=1,
)

plt.plot(np.arange(0.2,5,0.1), np.arange(0.2,5,0.1), 'r--', alpha=0.5)
plt.ylabel('ComCam PSF FWHM @500nm AM=1')
plt.xlabel('RINGSS Seeing')
plt.xlim(0.2,3.0)
plt.ylim(0.2,3.0)
plt.title(f"ComCam Seeing vs RINGSS during ComCam On-Sky")
plt.legend()
plt.grid(True)
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()

In [ ]:
# Plot the time vs the specified column
plt.figure(figsize=(10, 6))
plt.plot(merged_table["see"], merged_table["fwhm_zenith_500nm"],'.', color='0.5',zorder=0, alpha=0.5)
sns.kdeplot(
    x=merged_table["see"], 
    y=merged_table["fwhm_zenith_500nm"], 
    levels=10,  # Number of contour levels
    cmap="Reds",  # Color map for contours
    alpha=0.6,
    zorder=1,
)

plt.plot(np.arange(0.2,5,0.1), np.arange(0.2,5,0.1), 'r--', alpha=0.5)
plt.ylabel('ComCam PSF FWHM @500nm AM=1')
plt.xlabel('RINGSS Seeing')
plt.xlim(0.2,3.0)
plt.ylim(0.2,3.0)
plt.title(f"ComCam Seeing vs RINGSS during ComCam On-Sky")
plt.grid(True)
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()

In [ ]:
# Plot histogram using Freedman-Diaconis rule
plt.figure()
plt.xlabel('Seeing')
plt.ylabel("Frequency")
plt.title(f"Difference in quadrature of ComCam FWHM \n and RINGSS Seeing during ComCam On-Sky")
plt.grid(True, linestyle="--", alpha=0.6, zorder=0)
plt.hist(np.sqrt(sqr_diff[sqr_diff>0]), bins='fd', edgecolor='0.1', zorder=2,alpha=0.5)
plt.legend()
plt.show()

In [ ]:
# Plot the time vs the specified column
plt.figure(figsize=(10, 6))
plt.plot(merged_table["fwhm_zenith_500nm"], sqr_diff,'.', color='0.5',zorder=0, alpha=0.5)
sns.kdeplot(
    x=merged_table["fwhm_zenith_500nm"], 
    y=sqr_diff, 
    levels=10,  # Number of contour levels
    cmap="Reds",  # Color map for contours
    alpha=0.6,
    zorder=1,
)

plt.ylabel('Squared Diff ComCam - RINGSS')
plt.xlabel('ComCam FWHM zenith 500nm')
plt.title(f"ComCam PSF FWHM vs Squared Difference with ComCam")
plt.legend()
plt.grid(True)
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()

In [ ]:
# Plot the time vs the specified column
plt.figure(figsize=(10, 6))
plt.plot(merged_table["see"], sqr_diff,'.', color='0.5',zorder=0, alpha=0.5)
sns.kdeplot(
    x=merged_table["see"], 
    y=sqr_diff, 
    levels=10,  # Number of contour levels
    cmap="Reds",  # Color map for contours
    alpha=0.6,
    zorder=1,
)

plt.ylabel('ComCam Seeing - RINGSS Seeing')
plt.xlabel('RINGSS Seeing')
plt.title(f"RINGSS Seeing vs Squared Difference with ComCam")
plt.legend()
plt.grid(True)
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()

In [ ]:
diff = merged_table["fwhm_zenith_500nm"] - merged_table["see"]

In [ ]:
# Plot the time vs the specified column
plt.figure(figsize=(10, 6))
plt.plot(merged_table["see"], diff,'.', color='0.5',zorder=0, alpha=0.5)
sns.kdeplot(
    x=merged_table["see"], 
    y=diff, 
    levels=10,  # Number of contour levels
    cmap="Reds",  # Color map for contours
    alpha=0.6,
    zorder=1,
)

plt.ylabel('ComCam Seeing - RINGSS Seeing', size='large')
plt.xlabel('RINGSS Seeing', size='large')
plt.title(f"Differennce Between ComCam and RINGSS during ComCam On-Sky")
plt.legend()
plt.grid(True)
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()

There seems to be an issue with our PSF measurements in the ConsDB as a function of seeing. That is, we are underestimating the PSF FWHM in the consDB at high seeing. Let's see what the DRP has. 

In [ ]:
from lsst.daf.butler import Butler

collection='LSSTComCam/runs/DRP/DP1/w_2025_05/DM-48666'
butler = Butler('/sdf/group/rubin/repo/main', collections=collection)
df = butler.get("ccdVisitTable")

In [ ]:
grouped_df = (
    df.groupby(["visitId"])
    .agg(lambda x: x.median() if np.issubdtype(x.dtype, np.number) else x.iloc[0])  
    .reset_index()
)
grouped_df = grouped_df.rename(columns=lambda x: f"drp_{x}")
grouped_df.rename(columns={"drp_visitId": "visit_id"})

In [ ]:
drp_table = Table.from_pandas(grouped_df)
drp_table.rename_column("drp_visitId", "visit_id")

In [ ]:
merged_drp_table = join(merged_table, drp_table, keys=["visit_id"], join_type="inner")

In [ ]:
# Convert PSF sigma to FWHM
sig2fwhm = 2 * np.sqrt(2 * np.log(2))
pixel_scale = 0.2  # arcsec / pixel
merged_drp_table["drp_psf_fwhm"] = merged_drp_table["drp_psfSigma"] * sig2fwhm * pixel_scale

# Add the FWHM at zenith at 500nm
merged_drp_table["drp_fwhm_zenith_500nm"] = [
    fwhm * getBandpassSeeingCorrection(filt) * getAirmassSeeingCorrection(airmass)
    for fwhm, filt, airmass in zip(merged_drp_table["drp_psf_fwhm"], merged_drp_table["physical_filter"], merged_drp_table["airmass"])
]

In [ ]:
# Plot the time vs the specified column
plt.figure(figsize=(10, 6))
plt.plot(merged_drp_table["fwhm_zenith_500nm"], merged_drp_table["drp_fwhm_zenith_500nm"] - merged_drp_table["fwhm_zenith_500nm"],'.', color='0.5',zorder=0, alpha=0.5)
sns.kdeplot(
    x=merged_drp_table["fwhm_zenith_500nm"], 
    y=merged_drp_table["drp_fwhm_zenith_500nm"] - merged_drp_table["fwhm_zenith_500nm"], 
    levels=10,  # Number of contour levels
    cmap="Reds",  # Color map for contours
    alpha=0.6,
    zorder=1,
)

plt.xlabel('QuickLook Seeing', size='large')
plt.ylabel('DRP - QuickLook Seeing', size='large')
plt.title(f"DRP vs QuickLook Seeing for ComCam on-sky")
plt.grid(True)
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()

In [ ]:
merged_drp_table.columns

In [ ]:
merged_drp_table['drp_seeing'] - merged_drp_table['drp_fwhm_zenith_500nm']

There's a value in the visit summary table called seeing? I'm not sure where this is coming from, but the distribution looks quite different than what RINGSS measured.

In [ ]:
# Plot the time vs the specified column
plt.figure(figsize=(10, 6))
plt.plot(merged_drp_table["drp_seeing"], merged_drp_table["drp_fwhm_zenith_500nm"] - merged_drp_table['drp_seeing'],'.', color='0.5',zorder=0, alpha=0.5)
sns.kdeplot(
    x=merged_drp_table["drp_seeing"], 
    y=merged_drp_table["drp_fwhm_zenith_500nm"] - merged_drp_table['drp_seeing'], 
    levels=10,  # Number of contour levels
    cmap="Reds",  # Color map for contours
    alpha=0.6,
    zorder=1,
)

plt.xlabel('DRP Seeing', size='large')
plt.ylabel('DRP FWHM zenith 500nm', size='large')
plt.title(f"DRP seeing vs FWHM zenith 500nm ComCam on-sky")
plt.grid(True)
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()

In [ ]:
# Plot the time vs the specified column
plt.figure(figsize=(10, 6))
plt.plot(merged_drp_table["drp_seeing"], merged_drp_table["see"],'.', color='0.5',zorder=0, alpha=0.5)
sns.kdeplot(
    x=merged_drp_table["drp_seeing"], 
    y=merged_drp_table["see"], 
    levels=10,  # Number of contour levels
    cmap="Reds",  # Color map for contours
    alpha=0.6,
    zorder=1,
)

plt.xlabel('DRP Seeing', size='large')
plt.ylabel('RINGSS Seeing', size='large')
plt.title(f"DRP seeing vs RINGSS seeing ComCam on-sky")
plt.grid(True)
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()

In [ ]:
# Plot the time vs the specified column
plt.figure(figsize=(10, 6))
plt.plot(merged_drp_table["drp_seeing"], merged_drp_table["see"]-merged_drp_table["drp_seeing"],'.', color='0.5',zorder=0, alpha=0.5)
sns.kdeplot(
    x=merged_drp_table["drp_seeing"], 
    y=merged_drp_table["see"]-merged_drp_table["drp_seeing"], 
    levels=10,  # Number of contour levels
    cmap="Reds",  # Color map for contours
    alpha=0.6,
    zorder=1,
)

plt.xlabel('DRP Seeing', size='large')
plt.ylabel('RINGSS - DRP Seeing', size='large')
plt.title(f"DRP seeing vs RINGSS seeing ComCam on-sky")
plt.grid(True)
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()